In [1]:
import folium
import numpy as np
import pandas as pd
from collections import namedtuple
import urllib
import urllib.request
import requests
import json
from google.transit import gtfs_realtime_pb2 as gtfs
from lxml import html

In [2]:
url = "http://gtfsrt.prod.obanyc.com/vehiclePositions?key=7ba40d81-d4da-41db-8832-3396a5572956"

In [3]:
tree = requests.get('http://www.easysurf.cc/M1.htm')

In [4]:
tree = html.fromstring(tree.content)

In [5]:
stop_code = tree.xpath('//center[3]/table/tbody/tr/td[3]/a/text()')

In [6]:
stops_df = pd.read_csv("https://openmobilitydata-data.s3-us-west-1.amazonaws.com/public/feeds/mta/82/20190705/original/stops.txt")

In [7]:
stops_df.head()

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,400001,4 AV/E 10 ST,NaN,40.731342,-73.990292,NaN,NaN,0,NaN
1,400002,4 AV/E 12 ST,NaN,40.732608,-73.989958,NaN,NaN,0,NaN
2,400003,4 AV/E 13 ST,NaN,40.733936,-73.989720,NaN,NaN,0,NaN
3,400007,PARK AV S/E 21 ST,NaN,40.738976,-73.987053,NaN,NaN,0,NaN
4,400026,MADISON AV/E 63 ST,NaN,40.766274,-73.969329,NaN,NaN,0,NaN


In [8]:
stop_list = []
for i in range(len(stop_code)):
    stop_list.append(int(stop_code[i]))

In [9]:
code_df = pd.DataFrame(stop_list)
code_df.rename(columns = {0:'stop_id'},inplace = True)

In [10]:
code_latlong_df = code_df.merge(stops_df, on= "stop_id")

In [11]:
code_latlong_df.head()

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,405284,MALCOLM X BL/W 146 ST,NaN,40.821082,-73.935896,NaN,NaN,0,NaN
1,400088,MALCOLM X BL/W 145 ST,NaN,40.820359,-73.936460,NaN,NaN,0,NaN
2,400089,MALCOLM X BL/W 142 ST,NaN,40.818247,-73.938003,NaN,NaN,0,NaN
3,400091,W 139 ST/MALCOLM X BL,NaN,40.816381,-73.938746,NaN,NaN,0,NaN
4,400092,5 AV/W 138 ST,NaN,40.815202,-73.935980,NaN,NaN,0,NaN


In [12]:
stop_latlong = []
for i in range(len(code_latlong_df)):
    stop_latlong.append([code_latlong_df['stop_lat'].iloc[i],code_latlong_df['stop_lon'].iloc[i]])

In [13]:
response = requests.get(url)
feed = gtfs.FeedMessage()
feed.ParseFromString(response.content)

271629

In [14]:
bus = []
for i in range(len(feed.entity)):
    if feed.entity[i].vehicle.trip.route_id == 'M1':
        bus.append([feed.entity[i].vehicle.position.latitude,feed.entity[i].vehicle.position.longitude])

In [15]:
import folium
import pandas as pd
latitude = 40.793943
longitude = -73.885880
icon_url = "https://img.icons8.com/dusk/2x/bus.png"
icon_stop_url = "https://image.flaticon.com/icons/svg/177/177813.svg"

In [16]:
m = folium.Map(
    location=[40.793943, -73.885880],
    zoom_start=12,
    tiles='OpenStreetMap'
)

for each in bus:
    icon_url = 'https://img.icons8.com/cotton/64/000000/school-bus-2.png'
    icon = folium.features.CustomIcon(icon_url,icon_size=(40, 40)) 
    folium.Marker(each,icon = icon).add_to(m)
# popup='<b>Timberline Lodge</b>'
for stop in stop_latlong:
    icon_stop_url = "https://image.flaticon.com/icons/svg/177/177813.svg"
    icon = folium.features.CustomIcon(icon_stop_url,icon_size=(15, 15)) 
    folium.Marker(stop,icon = icon).add_to(m)
my_PolyLine=folium.PolyLine(locations=stop_latlong,weight=5)
m.add_children(my_PolyLine)
m

/home/angel/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.


In [17]:
m.save('bus_line.html')